In [1]:
import glob
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
plt.rcParams['figure.figsize'] = [14, 7]

In [2]:
import import_ipynb
import drivers
import prepare

importing Jupyter notebook from drivers.ipynb
importing Jupyter notebook from prepare.ipynb


In [3]:
nifty = prepare.MergedDf()

In [4]:
nifty.describe()

,Open,High,Low,Close,Shares Traded,Turnover (Rs. Cr),P/E,P/B,Div Yield
count,5225.000000,5225.000000,5225.000000,5225.000000,5.225000e+03,5225.000000,5225.000000,5225.000000,5225.000000
mean,4931.530211,4964.677742,4892.778804,4929.321809,1.515437e+08,6250.129768,19.882100,3.539041,1.419129
std,3247.573752,3256.301398,3231.571643,3243.876880,1.191109e+08,4840.834747,4.159403,0.798190,0.400195
min,853.000000,877.000000,849.950000,854.200000,1.394931e+06,40.120000,10.680000,1.920000,0.590000
25%,1667.450000,1688.250000,1644.400000,1668.750000,6.926587e+07,2620.680000,17.010000,3.020000,1.160000
50%,4877.850000,4930.250000,4833.050000,4875.050000,1.305892e+08,5462.340000,19.940000,3.470000,1.320000
75%,7588.550000,7635.550000,7532.450000,7580.200000,1.900432e+08,8149.000000,22.660000,3.800000,1.540000
max,12274.900000,12293.900000,12252.750000,12271.800000,1.414837e+09,54081.530000,29.900000,6.550000,3.180000


In [5]:
nifty.loc['1999-01-04']

Open                      896.40
High                      905.45
Low                       895.75
Close                     897.80
Shares Traded        32224833.00
Turnover (Rs. Cr)         811.39
P/E                        11.72
P/B                         2.08
Div Yield                   1.81
Name: 1999-01-04 00:00:00, dtype: float64

In [6]:
params = drivers.Parameters(monthly_sip=100,
                            default_exposure=0.5,
                            green_pe=18,
                            greener_pe=14,
                            red_pe=29,
                            redder_pe=29)

In [7]:
e = drivers.EquityCorpus(0)

In [8]:
# debt = DebtCorpus()
# print(debt.Deposit(datetime(2000, 1, 1), 100))
# print(debt.Withdraw(datetime(2010, 1, 1), 100))
# debt.Get(datetime(2020, 1, 1))


# vanilla strategy with params

# monthly_sip = 100
# default_exposure = 0.5
# green_pe = 15
# red_pe = 28

# Every month, invest monthly_sip * default_exposure in index and invest monthly_sip * (1 - default_exposure) in debt.
# If nifty pe > red_pe, pull out all money from index to debt.
# if nifty pe < green_pe, pull out all money from debt to index.

In [9]:
nifty.index[-1].to_pydatetime()

datetime.datetime(2019, 12, 31, 0, 0)

In [10]:
def EvaluateStrategy(df, params):
    curr_month = -1
    e = drivers.EquityCorpus(df)
    d = drivers.DebtCorpus()
    total_invested = 0;
    for ind in df.index:
        if ind.month != curr_month:
            curr_month = ind.month
            index_sip = params.monthly_sip * params.default_exposure
            e.Deposit(ind, index_sip)
            debt_sip = params.monthly_sip * (1 - params.default_exposure)
            d.Deposit(ind, debt_sip)
            total_invested += params.monthly_sip
            current_pe = df['P/E'][ind]
            if (current_pe < params.green_pe):
                debt_funds = d.Get(ind)
                d.Withdraw(ind, debt_funds)
                e.Deposit(ind, debt_funds)
            elif (current_pe > params.red_pe):
                equity_funds = e.Get(ind)
                e.Withdraw(ind, equity_funds)
                d.Deposit(ind, equity_funds)
    end_date = df.index[-1].to_pydatetime()
    print('total_invested', total_invested, 'e.Get()', e.Get(end_date), 'd.Get()', d.Get(end_date))
    

In [11]:
EvaluateStrategy(nifty, params)

total_invested 25200 e.Get() 266.9844453634313 d.Get() 118062.59563482716
